<a href="https://colab.research.google.com/github/jang-jaehyuk/yoloproject/blob/master/xmltoyolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import

In [2]:
import os
import xml.etree.ElementTree as ET

## XML 포맷을 YOLO 포맷으로 변환하는 함수

In [34]:
def xml2yolo(xml_path, output_path, fierce = False):
    # input, output path 정의
    ANNO_path = xml_path        # xml 포맷으로 저장된 annotation이 저장된 경로
    OUTPUT_ANNO_path = output_path

    if not os.path.exists(OUTPUT_ANNO_path):                       # 저장할 경로가 생성되 있지 않다면 directory 생성
        os.mkdir(OUTPUT_ANNO_path)
    
    remove_cnt = 0                                            # size가 없어서 삭제된 개수
    ANNO_list = os.listdir(ANNO_path)                         #  XML 파일 전체 list

    for f in ANNO_list:                                       #  XML 파일 하나씩 작업   
        file_path = os.path.join(ANNO_path, f)                #  파일경로

        # ===== xml파일 parsing ===== # 
        tree = ET.parse(file_path)
        root = tree.getroot()

        file_name = root.find('filename').text                #  XML파일에 <filename> 정보 가져오기
        dog_name = root.find('object').find('name').text      #  개 이름 가져오기

        if fierce:
            class_name = 'fierce' if dog_name in [            # 만약 개이름이 리스트안에 있다면
                'Staffordshire Bull Terrier',                 # class_name 은 fierce
                'American Pit Bull Terrier',                  # 없다면 class_name은 normal
            ] else 'normal'                        

            class_id = 0 if class_name == 'normal' else 'fierce'  # class_name이 normal이면 id는 0 fierce면 fierce
        
        size = root.find('size')                              #  XML파일에 <size> 정보 가져오기

        if size is None:                                      # 사이즈가 없다면 삭제개수 + 1
            remove_cnt += 1
            continue

        img_width = int(size.find('width').text)              # <size>에 width 저장
        img_height = int(size.find('height').text)            # <size>에 height 저장

        with open(os.path.join(OUTPUT_ANNO_path, f'{file_name}.txt'), 'w') as f:
              # 오브젝트 정보
              obj = root.find('object')
              
              # bounding box 정보
              if obj.find('bndbox') is None:
                box = obj.find('bodybndbox')
              else: box = obj.find('bndbox')

              # ===== 바운딩박스 좌표 가져오기 ===== # 
              xmin = int(box.find('xmin').text)
              ymin = int(box.find('ymin').text)
              xmax = int(box.find('xmax').text)
              ymax = int(box.find('ymax').text)

              # ===== yolo형식에 맞게 계산 ===== # 
              x = round(((xmin + xmax) / 2)/img_width, 7)
              y = round(((ymin + ymax) / 2)/img_height, 7)
              width = round((xmax - xmin)/img_width, 7)
              height = round((ymax - ymin)/img_height, 7)

              # ===== yolo형식에 맞게 입력 ===== # 
              text = '{} {} {} {} {}'.format(class_id, x, y, width, height)

              f.write(text + '\n')

# YOLO포맷으로 변환
## annotation파일 경로, 결과를 저장할 파일, 맹견라벨링 할지 파라미터로 입력
xml파일명과 동일한파일명.txt로 labels폴더에 저장됨

In [36]:
xml2yolo("/content/drive/MyDrive/annotation", "/content/drive/MyDrive/labels", True)